In [421]:
#importing the necessary libraries
import numpy as np
import math

In [422]:
# unpickle method as given by https://www.cs.toronto.edu/~kriz/cifar.html
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [423]:
# defining the K as in K-Nearest Neighbors
k=8

In [424]:
# reading the test set
test=unpickle("cifar-10-batches-py/test_batch")

In [426]:
# reading the test data and labels in seperate variables
test_data=test[b'data']
test_labels=test[b'labels']

In [431]:
# initializing a list for all predicted labels for test data
knn_labels=[]

In [ ]:
# looping through every example in test data
for x in range(0,len(test_data)):
#   initializing a list to store the k nearest distances and labels
    distances=[]
#   looping throught the 6 batches and unpacking each of them to determine neighbors
    for i in range(1,6):
        b=unpickle("cifar-10-batches-py/data_batch_"+str(i))
#       looping through all the training examples to in the Ith batch
        for j in range(0,len(b[b'data'])):
#           calculating euclidian distance of Xth test example from Jth Training example in Ith batch
            distance=math.sqrt(np.sum(np.square(test_data[x]-b[b'data'][j])))
#           adding the calculated distance, along with the label of the jth example as a tupple
#           see sorted_add function
            distances=sorted_add(distances,(distance,b[b'labels'][j]),k)
#   flattening the list to apply list comprehension in the next step
    flat=list(sum(distances, ()))
#   selecting all the class labels i.e every odd element of flattened list
    lbls=[flat[i] for i in range(1,len(flat),2)]
#   finding the label that occurs most times in labels list (majority voting)
    pred=max(set(lbls), key=lbls.count)
#   adding the predicted label to a list
    knn_labels.append(pred)

In [444]:
# initializing correctly predicted counts for all labels
correct=[0,0,0,0,0,0,0,0,0,0]
# for every predicted label comparing it with the true label
for i in range(0,len(knn_labels)):
    if int(knn_labels[i]) == test_labels[i]:
#           counting the correctly predicted lables
            correct[test_labels[i]]+=1

In [456]:
# Model accuracy
class_acc=(np.array(correct)/1000)*100
print("Overall Model Accurary is", np.mean(class_acc))

Overall Model Accurary is 10.58


In [457]:
# sorted_arr takes 3 arguments
# 1. The list in which the item is needed to be add in
# 2. The item itself, here is a tuple for it eg. (distance, label)
# 3. The K, as in K-Nearest Neighbor (only K smallest elements will remain in the list at the end)
def sorted_add(ar,item,k):
#   adding first k elements directly into the array
    if len(ar)<k:
        ar.append(item)
        return ar
#   finding the tuple with max value of distance in it
    checker=max([ar[i] for i in range(1,len(ar),2)])
#   checking if the distance in given item is smaller that the current max
    if (int(item[1])<int(checker[1])):
#       if it is, then taking the index of the current max
        index=ar.index(checker)
#       and deleting that element from the list
        del ar[index]
#       then adding the given element to the list
        ar.append(item)
#   returning the array, Note: this array itself will be unsorted,
#   because order in the k-nearest neighbors does not matter for majority voting
    return ar